1. 1 слой 26 нейронов (сначала передать туда матрицу декодера, потом инициализировать рандомно) 
2. 2 слоя по 31 нейрона
3. 

1. бинарный вход перевести в вещественный в [0,1]


In [ ]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
# !mkdir -p drive
# !google-drive-ocamlfuse drive

In [ ]:
import os 
import sys
import pandas as pd
import numpy as np

from keras import models
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Concatenate
# from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint


from sklearn import model_selection
from __future__ import absolute_import

# os.chdir("/content/drive/Hamming")
# sys.path.append("Hamming")
# import functions as f
# import model_lib 

In [ ]:
!ls

In [ ]:
# !git pull

In [ ]:
TRAIN_PATH = './dataset_files/hamming.txt'  # "/content/drive/Hamming/dataset_files/hamming.txt"

COLUMN_NAMES = ['id', 'plainword', 'codeword', 
                'id_error', 'bin_error', 'defective_codeword']

In [ ]:
def hamming_distance(first: str, second: str) -> int:
    return len([1 for (x, y) in zip(first, second) if x != y])

In [ ]:
def loss_hamming(y, y_pred):  # , w):
    return sum(1 for (a,b) in zip(y, y_pred) if y != y_pred) / y.shape[0] # надо ли делить????

In [ ]:
def load_data():
    data = pd.read_csv(TRAIN_PATH, sep=';', names=COLUMN_NAMES)

    # make features
#     data['dec_defective_codeword'] = data['defective_codeword'][:].apply(lambda x: int(x, 2))
#     for j in range(len(data['defective_codeword'][0])):
#         data['bin_' + str(j)] = data['defective_codeword'][:].apply(lambda x: int(x[j]))
#     data['dec_plainword'] = data['plainword'][:].apply(lambda x: int(x, 2))
    return data

In [ ]:
def make_features():
    data['dec_defective_codeword'] = data['defective_codeword'][:].apply(lambda x: int(x, 2))

    for j in range(len(data['codeword'][0])):
        data['cod_' + str(j)] = data['codeword'][:].apply(lambda x: int(x[j]))

    for j in range(len(data['defective_codeword'][0])):
        data['def_' + str(j)] = data['defective_codeword'][:].apply(lambda x: int(x[j]))

    for j in range(len(data['bin_error'][0])):
        data['mask_' + str(j)] = data['bin_error'][:].apply(lambda x: int(x[j]))
        
    for j in range(len(data['plainword'][0])):
        data['pln_' + str(j)] = data['plainword'][:].apply(lambda x: int(x[j]))


In [ ]:
def split_data(test_size):
    train_data_defective, test_data_defective, train_labels_defective, test_labels_defective = \
    model_selection.train_test_split(data.loc[:, 'def_0':'def_30'], 
                                     data.loc[:, 'pln_0':'pln_25'],  # 'mask_0':'pln_25'
                                        test_size = test_size)
    
    train_data_plain, test_data_plain, train_labels_plain, test_labels_plain = \
    model_selection.train_test_split(data.loc[:, 'cod_0':'cod_30'], 
                                     data.loc[:, 'mask_0':'pln_25'], 
                                        test_size = test_size)
    
    train_data = np.vstack((np.array(train_data_defective), np.array(train_data_defective)))
    test_data = np.vstack((np.array(test_data_defective), np.array(test_data_defective)))
    train_labels = np.vstack((np.array(train_labels_defective), np.array(train_labels_defective)))
    test_labels = np.vstack((np.array(test_labels_defective), np.array(test_labels_defective)))
    
    return train_data, test_data, train_labels, test_labels

In [ ]:
data = load_data() #pd.read_csv(functions.TRAIN_PATH, sep=';', names = functions.COLUMN_NAMES)

In [ ]:
%%time
make_features()

In [ ]:
data.head()

In [ ]:
train_data, test_data, train_labels, test_labels = split_data(test_size=0.3)

In [ ]:
decoder = []
with open('./dataset_files/h.txt') as f:
    decoder = pd.read_csv(f, header=None, names=['h'], dtype=str) #, sep='\n')
for j in range(len(decoder['h'][0])):
    decoder['bin_' + str(j)] = decoder['h'][:].apply(lambda x: int(x[j]))
decoder = decoder.drop(['h'], axis=1)
decoder_np = decoder.as_matrix()
print(decoder_np[:6])

model = Sequential()
model.add(Dense(31, input_shape=(31,), 
#                 weights=decoder_np, 
                activation='relu', name='d1'))
# model.add(Dense(100, activation='relu', name='d2'))
# model.add(Dense(31, activation='softmax', name='d3'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy', # энтропия Шелдона / бинарная
              metrics=['accuracy', 'mse'])

MCP = ModelCheckpoint('best_model.hd5', save_best_only=True)

In [ ]:
# train_labels_categor = to_categorical(train_labels, num_classes=31)

model.fit(x=train_data, y=train_labels_categor, validation_split=0.2,  shuffle=True, epochs=100, callbacks=[MCP])

LSTM;

матринчая нейронная сеть, каждый слой - логические ячейки xor...
метрика хэмминга

подавать и правильные и неправильные


поставить керас
from keras... import
inp = Input(shape=(31,))
x = Dense(128, activation='elu')(inp)
x = Dense(128, activation='relu')(x)

test = Dense(.....)(x)

m = Dense(26, activation='пороговая функция типа step которой уже нет 'sigmoid')(x)

test = Dense(...)(test)
test= Dense(31....)(test)
m = Concatinate()([test,m])
          
model = Model(inp, m, 'm')
model.compile() , summary, fit

метки = бинарный код ошибки + информационное слово

лосс - бинарная кросс энтропия
оптимизатор - Адам
стартовая точноесть - 1е-5

In [ ]:
# from keras.layers import Conv1D, Reshape, Flatten

# inp = Input(shape=(31,))
# x = Dense(128, activation='relu',  name='d1')(inp)
# x = Dense(128, activation='relu',  name='d2')(x)

# t = Dense(128, activation='elu', name='t1')(x)
# t = Dense(128, activation='relu', name='t2')(t)
# t = Dense(31, activation='softmax', name='t3')(t)

# m = Dense(26, activation='sigmoid',  name='d3')(x)
# m = Concatenate()([t, m])

# model = Model(inp, m, 'm')
# model.compile(loss='binary_crossentropy', optimizer='Adam')
# model.summary()

In [ ]:
from keras.layers import Conv1D, Reshape, Flatten, MaxPooling1D, UpSampling1D
from keras.layers.recurrent import LSTM

inp = Input(shape=(31,))
x = Dense(1024, activation='relu')(inp)
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='elu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)

m = Dense(26, activation='sigmoid',  name='d3')(x)

model = Model(inp, m, 'm')
model.compile(loss='binary_crossentropy', optimizer='Adam')
model.summary()

In [ ]:
model.fit(x=train_data, y=train_labels, validation_split=0.2,  shuffle=True, epochs=5, batch_size=256)#, callbacks=[MCP])

Проверять на сокльки сэмплах совпало значение точно